In [1]:
from collections import OrderedDict
from typing import Tuple, Union

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch import nn

from datacat4ml.const import *
import ipdb

# SMILES Tokenizer

In [2]:
import re

class SMILESTokenizer:

    """
    Shared by Markus.
    
    """

    def __init__(self):
        self.pattern = r"(\[[^\]]+]|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\\\|\/|:|~|@|\?|>|\*|\!|\$|\%[0-9]{2}|[0-9])"
        self.vocab = {}
        self.inv_vocab = {}
        self.pad_token = '<PAD>'
        self.unk_token = '<UNK>'
        self.start_token = '<START>'
        self.end_token = '<END>'
        self.max_len = None
    def tokenize(self, smiles):
        """Tokenizes a SMILES string using the predefined regular expression."""
        return re.findall(self.pattern, smiles)
    def build_vocab(self, smiles_list):
        """Builds vocabulary from a list of SMILES strings."""
        all_tokens = set()
        for smiles in smiles_list:
            tokens = self.tokenize(smiles)
            all_tokens.update(tokens)
        tokens = [self.pad_token, self.unk_token, self.start_token, self.end_token]
        all_tokens = sorted(all_tokens)
        all_tokens = tokens + all_tokens
        self.vocab = {token: idx for idx, token in enumerate(all_tokens)}
        self.inv_vocab = {idx: token for token, idx in self.vocab.items()}
    def encode(self, smiles, max_len=None):
        """Encodes a SMILES string into a list of token indices, optionally padding to max_len."""
        tokens = self.tokenize(smiles)
        tokens = [self.start_token] + tokens + [self.end_token]
        token_ids = [self.vocab.get(token, self.vocab[self.unk_token]) for token in tokens]
        if max_len:
            token_ids = token_ids[:max_len] + [self.vocab[self.pad_token]] * max(0, max_len - len(token_ids))
        return token_ids
    def decode(self, token_ids):
        """Decodes a list of token indices back into a SMILES string."""
        tokens = [self.inv_vocab.get(token_id, self.unk_token) for token_id in token_ids]
        tokens = [token for token in tokens if token not in [self.start_token, self.end_token, self.pad_token]]
        return ''.join(tokens)
    def vocab_size(self):
        """Returns the size of the vocabulary."""
        return len(self.vocab)
    def pad_sequence(self, sequence, max_len):
        """Pads a sequence to the maximum length."""
        return sequence[:max_len] + [self.vocab[self.pad_token]] * max(0, max_len - len(sequence))

# CLIP/clip/model.py

In [4]:
class LayerNorm(nn.LayerNorm):
    """Subclass torch's LayerNorm to handle fp16."""

    def forward(self, x: torch.Tensor):
        orig_type = x.dtype
        ret = super().forward(x.type(torch.float32))
        return ret.type(orig_type)


class QuickGELU(nn.Module):
    def forward(self, x: torch.Tensor):
        return x * torch.sigmoid(1.702 * x)
    
class ResidualAttentionBlock(nn.Module):
    def __init__(self, d_model: int, n_head: int, attn_mask: torch.Tensor = None):
        super().__init__()

        self.attn = nn.MultiheadAttention(d_model, n_head)
        self.ln_1 = LayerNorm(d_model)
        self.mlp = nn.Sequential(OrderedDict([
            ("c_fc", nn.Linear(d_model, d_model * 4)),
            ("gelu", QuickGELU()),
            ("c_proj", nn.Linear(d_model * 4, d_model))
        ]))
        self.ln_2 = LayerNorm(d_model)
        self.attn_mask = attn_mask

    def attention(self, x: torch.Tensor):
        self.attn_mask = self.attn_mask.to(dtype=x.dtype, device=x.device) if self.attn_mask is not None else None
        return self.attn(x, x, x, need_weights=False, attn_mask=self.attn_mask)[0]

    def forward(self, x: torch.Tensor):
        x = x + self.attention(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x
    
class Transformer(nn.Module):
    def __init__(self, width: int, layers: int, heads: int, attn_mask: torch.Tensor = None):
        super().__init__()
        self.width = width
        self.layers = layers
        self.resblocks = nn.Sequential(*[ResidualAttentionBlock(width, heads, attn_mask) for _ in range(layers)])

    def forward(self, x: torch.Tensor):
        return self.resblocks(x)


In [5]:
class CLIPGPCR(nn.Module):
    def __init__(self,
                 embed_dim: int,
                 # molecule
                 smiles_vocab_size: int,

                 # assay text
                 context_length: int,
                 vocab_size: int,
                 transformer_width: int,
                 transformer_heads: int,
                 transformer_layers: int,
                 ):
        super().__init__()

        self.context_length = context_length

        # Initialize the SMILES tokenizer
        self.smiles_tokenizer = SMILESTokenizer()
        self.smiles_vocab_size = smiles_vocab_size
        self.smiles_embedding = nn.Embedding(smiles_vocab_size, transformer_width)
        self.smiles_projection = nn.Parameter(torch.empty(transformer_width, embed_dim))

        self.transformer = Transformer(
            width = transformer_width,
            layers = transformer_layers,
            heads = transformer_heads,
            attn_mask= self.build_attention_mask()
        )

        self. vocab_size = vocab_size
        self.token_embedding = nn.Embedding(vocab_size, transformer_width)
        self.positional_embedding = nn.Parameter(torch.empty(self.context_length, transformer_width))
        self.ln_final = LayerNorm(transformer_width)

        self.text_projection = nn.Parameter(torch.empty(transformer_width, embed_dim))
        self.logit_scale = nn.Parameter(torch.ones([]) *np.log(1 / 0.07))

        self.initizalize_parameters()
    
    def initialize_parameters(self):
        nn.init.normal_(self.token_embedding.weight, std=0.02)
        nn.init.normal_(self.positional_embedding, std=0.01)
        nn.init.normal_(self.smiles_embedding.weight, std=0.02)
        nn.init.normal_(self.smiles_projection, std=0.01)

        proj_std = (self.transformer.width ** -0.5) * ((2 * self.transformer.layers) ** -0.5)
        attn_std = self. transformer.width ** -0.5
        fc_std = (2 * self.transformer.width) ** -0.5
        for block in self.transformer.resblocks:
            nn.init.normal_(block.attn.in_proj_weight, std=attn_std)
            nn.init.normal_(block.attn.out_proj.weight, std=attn_std)
            nn.init.normal_(block.mlp.c_fc.weight, std=fc_std)
            nn.init.normal_(block.mlp.c_proj.weight, std=proj_std)
        
        if self.text_projection is not None:
            nn.init.normal_(self.text_projection, std=self.transformer.width ** -0.5)
    
    def build_attention_mask(self):
        # lazily create causal attention mask, with full attention between the smiles tokens
        # pytorch uses additive attention mask; fill with -inf
        mask = torch.empty(self.context_length, self.context_length)
        mask.fill_(float("_inf"))
        mask.triu_(1) # zero out the lower diagonal
        return mask
    
    @property
    def dtype(self):
        return self.smiles_embedding.weight.dtype
    
    def encode_smiles(self, smiles):

        token_ids = [self.smiles_tokenizer.encode(s) for s in smiles]
        token_ids = torch.tensor(token_ids).to(self.dtype)
        x = self.smiles_embedding(token_ids)
        x = x.mean(dim=1) # average the embeddings
        x = x @ self.smiles_projection

        return x

    def encode_text(self, text):
        x = self.token_embedding(text).type(self.dtype)
        x = x + self.positional_embedding.type(self.dtype)
        x = x.permute(1, 0, 2)
        x = self.transformer(x)
        x = x.permute(1, 0, 2)
        x = self.ln_final(x).type(self.dtype)
        x = x[torch.arange(x.shape[0]), text.argmax(dim=1)] @ self.text_projection

        return x

    def forward(self, smiles, text):
        smiles_features = self.encode_smiles(smiles)
        text_features = self.encode_text(text)

        # normalize features
        smiles_features = smiles_features / smiles_features.norm(dim=-1, keepdim=True)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)

        # cosine similarity as logits
        logit_scale = self.logit_scale.exp()
        logits_per_smiles = logit_scale * smiles_features @ text_features.t()
        logits_per_text = logits_per_smiles.t()

        return logits_per_smiles, logits_per_text

In [ ]:
def convert_weights(model: nn.Module):
    """Convert applicable model parameters to fp16"""

    def _convert_weights_to_fp16(l):
        if isinstance(l, (nn.Conv1d, nn.Conv2d, nn.Linear)):
            l.weight.data = l.weight.data.half()
            if l.bias is not None:
                l.bias.data = l.bias.data.half()

        if isinstance(l, nn.MultiheadAttention):
            for attr in [*[f"{s}_proj_weight" for s in ["in", "q", "k", "v"]], "in_proj_bias", "bias_k", "bias_v"]:
                tensor = getattr(l, attr)
                if tensor is not None:
                    tensor.data = tensor.data.half()

        for name in ["text_projection", "proj"]:
            if hasattr(l, name):
                attr = getattr(l, name)
                if attr is not None:
                    attr.data = attr.data.half()

    model.apply(_convert_weights_to_fp16)

def build_model(state_dict:dict):
    # Extract relevant parameters from the state_dict
    embed_dim = state_dict["text_projection"].shape[1]
    context_length = state_dict["positional_embedding"].shape[0]
    vocab_size = state_dict["token_embedding.weight"].shape[0]
    transformer_width = state_dict["ln_final.weight"].shape[0]
    transformer_heads = transformer_width // 64
    transformer_layers = len(set(k.split(".")[2] for k in state_dict if k.startswith("transformer.resblocks")))

    # SMILES specific parameters
    smiles_vocab_size = state_dict["smiles_embedding.weight"].shape[0]

    # Initialize the modified CLIP model with SMILES and text handling
    model = CLIPGPCR(
        embed_dim,
        smiles_vocab_size,
        context_length,
        vocab_size,
        transformer_width,
        transformer_heads,
        transformer_layers
    )

    for key in ["smiles_vocab_size", "context_length", "vocab_size"]: # "input_resolution" --> "smiles_vocab_size"
        if key in state_dict:
            del state_dict[key]
    
    convert_weights(model)
    model.load_state_dict(state_dict)
    
    return model.eval()
    

# Train CLIPGPCR

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

# import CLIPGPCR
# import text tokenizer
from datacat4ml.Scripts.model_dev.repo_clip.clip.simple_tokenizer import SimpleTokenizer
# import smiles tokenizer

## Prepare the data

In [12]:
with open("gpcr_assay_descs.txt", "r") as f:
    gpcr_texts = f.readlines()

with open("gpcr_smis.txt", "r") as f:
    gpcr_smiles = f.readlines()

# load the training data into a pandas dataframe
datapath = os.path.join(CURA_GPCR_DATASETS_DIR, "cls")
# load all files with a suffix of '_curated.csv'
files = [f for f in os.listdir(datapath) if f.endswith("_curated.csv")]
# load all the files into a single dataframe adn drop column 'Unnamed: 0'
df = pd.concat([pd.read_csv(os.path.join(datapath, f)) for f in files], ignore_index=True).drop(columns="Unnamed: 0")

In [13]:
df

,assay_id,assay_chembl_id,tid,target_chembl_id,standard_type,standard_relation,standard_value,standard_units,pchembl_value,assay_type,...,pStandard_value,max_num_atoms,max_molecular_weight,activity_string,activity,threshold,target,effect,assay,std_type
0,200520,CHEMBL801203,11261,CHEMBL2028,Ki,=,2660.0,nM,5.58,B,...,5.575118363368933,64,882.116,weak active,1.0,5.565998,CHEMBL2028,None,None,Ki
1,200520,CHEMBL801203,11261,CHEMBL2028,Ki,=,5250.0,nM,5.28,B,...,5.279840696594043,64,882.116,inactive,0.0,5.565998,CHEMBL2028,None,None,Ki
2,200520,CHEMBL801203,11261,CHEMBL2028,Ki,=,3750.0,nM,5.43,B,...,5.425968732272281,64,882.116,inactive,0.0,5.565998,CHEMBL2028,None,None,Ki
3,200506,CHEMBL803851,11261,CHEMBL2028,Ki,=,2574.0,nM,5.59,B,...,5.589391457431632,64,882.116,weak active,1.0,5.565998,CHEMBL2028,None,None,Ki
4,200506,CHEMBL803851,11261,CHEMBL2028,Ki,=,3614.0,nM,5.44,B,...,5.442011851775087,64,882.116,inactive,0.0,5.565998,CHEMBL2028,None,None,Ki
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289750,1804125,CHEMBL4276417,101356,CHEMBL5852,EC50,=,2747.0,nM,5.56,F,...,5.561141340579438,65,897.095,weak active,1.0,5.474955,CHEMBL5852,None,None,EC50
289751,1804125,CHEMBL4276417,101356,CHEMBL5852,EC50,>,100000.0,nM,None,F,...,4.0,65,897.095,inactive,0.0,5.474955,CHEMBL5852,None,None,EC50
289752,1804125,CHEMBL4276417,101356,CHEMBL5852,EC50,>,100000.0,nM,None,F,...,4.0,65,897.095,inactive,0.0,5.474955,CHEMBL5852,None,None,EC50
289753,1804125,CHEMBL4276417,101356,CHEMBL5852,EC50,>,100000.0,nM,None,F,...,4.0,65,897.095,inactive,0.0,5.474955,CHEMBL5852,None,None,EC50


In [9]:
# initialize tokenizer
text_tokenizer = SimpleTokenizer()
smiles_tokenizer = SMILESTokenizer()
smiles_tokenizer.build_vocab(gpcr_smiles) # make sure to build the vocabulary from your dataset

## Modify the training loop

In [14]:
# Initialize the model
model = CLIPGPCR(
    embed_dim=512,
    smiles_vocab_size=smiles_tokenizer.vocab_size(),
    context_length=77,
    vocab_size=len(text_tokenizer.encode),
    transformer_width=512,
    transformer_heads=8,
    transformer_layers=12
)

# set up the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(num_epochs):
    for smiles, text in dataloader:  # Replace with your actual dataloader
        optimizer.zero_grad()

        # Tokenize inputs
        smiles_ids = [smiles_tokenizer.encode(s) for s in smiles]
        text_ids = text_tokenizer.encode(text)

        smiles_ids = torch.tensor(smiles_ids).to(device)
        text_ids = torch.tensor(text_ids).to(device)

        # Forward pass
        logits_per_smiles, logits_per_text = model(smiles_ids, text_ids)

        # Compute loss
        labels = torch.arange(len(smiles)).to(device)
        loss = (criterion(logits_per_smiles, labels) + criterion(logits_per_text, labels)) / 2

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Save the trained model
torch.save(model.state_dict(), "clip_smiles_text_model.pth")

TypeError: object of type 'method' has no len()

## Save the model

# CLIP/clip/clip.py

The module merely loads the pre-trained clip models, without incorporating the training steps.

In [15]:
from packaging import version
from datacat4ml.Scripts.model_dev.repo_clip.clip.simple_tokenizer import SimpleTokenizer as _Tokenizer

# Usage